In [78]:
import pandas as pd
import numpy as np
import re
import os
from os import listdir
from os.path import isfile, join
import data_helpers
from hanziconv import HanziConv

In [79]:
weibo_m_o = pd.read_csv('./data/Weibo_M.csv')
weibo_o = pd.read_csv('./data/Weibo.csv')
wechat_o = pd.read_csv('./data/Wechat.csv')
news_forum_o = pd.read_csv('./data/News_forum.csv')

# Sentiment data

In [80]:
weibo_m_s = weibo_m_o[['COMMENT','SENTIMENT']].dropna(axis=0, how = 'any')
weibo_s = weibo_o[['COMMENT','SENTIMENT']].dropna(axis=0, how = 'any')
wechat_s = wechat_o[['CONTENT','SENTIMENT']].dropna(axis=0, how = 'any')

In [81]:
weibo_m_s.columns = ['CONTENT','SENTIMENT']
weibo_s.columns = ['CONTENT','SENTIMENT']

In [82]:
data = pd.concat([weibo_m_s,weibo_s,wechat_s])

In [83]:
data['CONTENT'] = data['CONTENT'].str.replace('\n', '')
data = data.reset_index(drop=True)

In [84]:
length = []
for i in range(len(data)):
    data['CONTENT'][i] = HanziConv.toSimplified(data['CONTENT'][i]).encode('utf-8')
    length.append(len(data['CONTENT'][i]))
    if data['SENTIMENT'][i] == 'Irrelevant':
        data['SENTIMENT'][i] = 'Neutral'
length = np.array(length)

In [86]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(data, test_size=0.2, random_state = 100)

In [91]:
testing_folder = './data/processed/testing/'
if not os.path.exists(testing_folder):
    os.makedirs(testing_folder)
for the_file in os.listdir(testing_folder):
    file_path = os.path.join(testing_folder, the_file)
    try:
        if os.path.isfile(file_path):
            os.unlink(file_path)
        #elif os.path.isdir(file_path): shutil.rmtree(file_path)
    except Exception as e:
        print(e)
vali_folder = './data/processed/validation/'
if not os.path.exists(vali_folder):
    os.makedirs(vali_folder)
for the_file in os.listdir(vali_folder):
    file_path = os.path.join(vali_folder, the_file)
    try:
        if os.path.isfile(file_path):
            os.unlink(file_path)
        #elif os.path.isdir(file_path): shutil.rmtree(file_path)
    except Exception as e:
        print(e)

In [92]:
for x in data['SENTIMENT'].unique():
    np.savetxt(vali_folder+ x.replace('/',' ') +'.utf8',\
                   train[train['SENTIMENT'] == x]['CONTENT'].values, fmt='%s')
for x in data['SENTIMENT'].unique():
    np.savetxt(testing_folder+ x.replace('/',' ') +'.utf8',\
                   test[test['SENTIMENT'] == x]['CONTENT'].values, fmt='%s')

In [88]:
train['SENTIMENT'].value_counts()

Neutral     11928
Positive      360
Negative      210
Name: SENTIMENT, dtype: int64

In [89]:
test['SENTIMENT'].value_counts()

Neutral     2943
Positive     105
Negative      77
Name: SENTIMENT, dtype: int64

In [1]:
from gensim.models import Word2Vec
w2vModel = Word2Vec.load('runs/trained_word2vec.model')

In [2]:
len(w2vModel.wv.vocab)

3577

In [68]:
from gensim.models import KeyedVectors
w2vModel = KeyedVectors.load_word2vec_format('runs/wiki.zh.vec')

In [53]:
vocab_obj = w2vModel.wv.vocab
word = u'河北'
print word
print vocab_obj[word].index

河北
3038


In [3]:
dianping_json = pd.read_table('./data/reviews.txt')

In [31]:
x = ''
for i in range(len(dianping_json)):
    x += (dianping_json.loc[i][0].split('^')[1]+'\n')

In [34]:
dianping = pd.read_json(x, lines=True)

ValueError: Unexpected character found when decoding object value

In [ ]:
import json

In [33]:
data = json.loads(x)

ValueError: Extra data: line 2 column 2 - line 4422474 column 1 (char 571 - 2724105557)

In [36]:
print '\n'

In [25]:
d

[u'content',
 u'atmosphere',
 u'dishes',
 u'service',
 u'restId',
 u'userId',
 u'waiting',
 u'environment',
 u'rate',
 u'cost',
 u'srcType',
 u'time',
 u'flavor',
 u'special',
 u'stage']